In [1]:
import ee
import geemap
import importlib

In [2]:
Map = geemap.Map()

In [3]:
ee.Initialize()

In [4]:
import utils.ee_utils as ee_utils

In [5]:
# Reload the module
importlib.reload(ee_utils)

<module 'utils.ee_utils' from 'd:\\python\\CropMapping\\dataset\\utils\\ee_utils.py'>

In [6]:

# Define area of interest.
roi = ee.Geometry.Polygon(
                                [[[1.2897224677359675, 48.38400739957012],
                                [1.2897224677359675, 47.93697958008443],
                                [3.2068367255484675, 47.93697958008443],
                                [3.2068367255484675, 48.38400739957012]]])

# Call function with defined start and end dates and area of interest.
cloudless_collection = ee_utils.create_collection(roi, '2019-05-01', '2019-06-30')

# Define visualization parameters.
visualization = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2'],
}

# Add layer to map.
Map.addLayer(cloudless_collection.median().clip(roi), visualization, 'Cloudless Collection')
Map.centerObject(roi, 10)

In [7]:
s1_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
          .filterDate('2019-05-01', '2019-06-30') \
          .filterBounds(roi) \
          .filter(ee.Filter.eq('resolution','H')) \
          .filter(ee.Filter.eq('instrumentMode','IW'))\
          .filter(ee.Filter.contains('.geo', roi)) \
          .filterMetadata('orbitProperties_pass', 'equals', "ASCENDING") 


In [8]:
s1_collection.size().getInfo()

5

In [9]:
ee_utils.sen1_print(s1_collection)

,Property,0,1,2,3,4
0,system:time_start,2019-05-04,2019-05-16,2019-05-28,2019-06-09,2019-06-21
1,orbitProperties_pass,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING
2,resolution,H,H,H,H,H
3,resolution_meters,10,10,10,10,10
4,platform_number,A,A,A,A,A
5,productType,GRD,GRD,GRD,GRD,GRD
6,orbitNumber_start,27081,27256,27431,27606,27781
7,orbitNumber_stop,27081,27256,27431,27606,27781
8,transmitterReceiverPolarisation,"['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']"
9,system:band_names,"['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']"


In [ ]:
s1_vis = {'min': -25, 'max': 5, 'bands': ['VV', 'VH', 'VV_VH_ratio']} # R: VV, G: VH, B: ratio
s1_collection = s1_collection.map(ee_utils.sen1_add_ratio)
s1_img = s1_collection.median().clip(roi)

false_color = {
    'bands': ['VV', 'VH', 'VV_VH_ratio'],
    'min': [-25, -35, 0],
    'max': [0, -5, 10]
}


Map.addLayer(s1_img, false_color, 'S1 Collection')
Map.centerObject(roi, 10)
Map

Map(center=[48.16415370878537, 2.248279596642088], controls=(WidgetControl(options=['position', 'transparent_b…